In [1]:
from unsloth import FastModel
import torch
import torch.nn as nn
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # model_name = "unsloth/Qwen3-8B-Base-unsloth-bnb-4bit",
    # model_name="unsloth/gemma-3-12b-pt",
    # model_name="unsloth/gemma-3-4b-pt",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-03 16:23:50 [__init__.py:235] Automatically detected platform cuda.
==((====))==  Unsloth 2025.7.11: Fast Qwen3 patching. Transformers: 4.54.1. vLLM: 0.10.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.635 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(2, 4096)
    (layers): ModuleList(
      (0): Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm

In [2]:
# save the lm_head for yes and no
index = [tokenizer.encode(" No")[0], tokenizer.encode(" Yes")[0]]
print(index)
torch.save(model.lm_head.weight[index], './Model/Gwen8B_lm_head.pth')

[2308, 7414]


In [1]:
from collections import defaultdict
from utility import group_examples_by_rule
import pandas as pd
df = pd.read_csv("Data/Data1/train.csv")
groups = group_examples_by_rule(df)

In [2]:
from utility import TTTDataset
dataset = TTTDataset(df, groups, None)

In [3]:
out = dataset._build_prompt(df.iloc[0])

In [6]:
text = """/r/krat0m <-subscribe for free kilo of strong kratom! Tonight at 8pm is drawing. 

Enter more than once!

Its a great sub too! Learn and have fun! Please help out. Its super new. But will be loaded with great content!"""

In [5]:
print(out[0])

You are given a comment on reddit. Your task is to classify if it violates the given rule.
Subreddit: r/Futurology
Rule: No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.
Comment: In the event you get weak or gentle erections, you are most likely battling with ED. Erectile dysfunction is more frequent than you can imagine. ED makes it impossible for you to satisfy the girl sexually and this could be a huge blow to your confidence.
<URL_levitra.global>
Violation: Yes
Comment: /r/krat0m <-subscribe for free kilo of strong kratom! Tonight at 8pm is drawing. 

Enter more than once!

Its a great sub too! Learn and have fun! Please help out. Its super new. But will be loaded with great content!
Violation: No
Comment: Banks don't want you to know this! Click here to know more!
Violation:
